In [14]:
import matplotlib.pylab as plt
import seaborn as sns
sns.despine()

import pandas as pd
import requests
import numpy as np
import io
from datetime import datetime
import os
pd.set_option('display.max_colwidth', 500)

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.layers.advanced_activations import *
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D
from keras.layers.recurrent import LSTM, GRU
from keras import regularizers

<Figure size 432x288 with 0 Axes>

In [3]:
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b
 
def create_Xt_Yt(X, y, percentage=0.9):
    p = int(len(X) * percentage)
    X_train = X[0:p]
    Y_train = y[0:p]
     
    X_train, Y_train = shuffle_in_unison(X_train, Y_train)
 
    X_test = X[p:]
    Y_test = y[p:]

    return X_train, X_test, Y_train, Y_test

In [22]:
## Data source: https://www.alphavantage.co/documentation/

#function = 'TIME_SERIES_INTRADAY'
function = 'TIME_SERIES_INTRADAY_EXTENDED'
symbol = 'IBM'
data_slice = 'year1month2'
api_key = 'MCFCVH33KFAYJBYH'
interval = '15min'
data_type = 'csv'

query = requests.get(
    f'https://www.alphavantage.co/query?function={function}&'
    f'symbol={symbol}&interval={interval}'
    f'&slice={data_slice}&apikey={api_key}'
    ).content

data = pd.read_csv(io.StringIO(query.decode('utf-8')))
data.head(10)

,time,open,high,low,close,volume
0,2021-02-23 18:30:00,120.70,120.7000,120.700,120.70,500
1,2021-02-23 17:15:00,120.56,120.5600,120.560,120.56,105
2,2021-02-23 17:00:00,120.70,120.7100,120.700,120.71,625
3,2021-02-23 16:45:00,120.54,120.5400,120.540,120.54,201
4,2021-02-23 16:15:00,120.71,120.7100,120.530,120.71,54856
5,2021-02-23 16:00:00,121.14,121.2100,120.680,120.71,422808
6,2021-02-23 15:45:00,121.13,121.2348,121.095,121.15,158899
7,2021-02-23 15:30:00,121.21,121.3500,121.110,121.12,137881
8,2021-02-23 15:15:00,121.21,121.2500,121.020,121.21,138390
9,2021-02-23 15:00:00,120.96,121.2270,120.890,121.21,147398


In [24]:
data.to_csv('../data/IBM_Feb_2021_15min.csv', index=False)

In [ ]:
data = pd.read_csv('AAPL.csv')[::-1]
data = data.ix[:, 'Adj Close'].tolist()

# Uncomment below to use price change time series
# data = data.ix[:, 'Adj Close'].pct_change().dropna().tolist()

plt.plot(data)
plt.show()

WINDOW = 30
EMB_SIZE = 1
STEP = 1
FORECAST = 5

# Straightforward way for creating time windows
X, Y = [], []
for i in range(0, len(data), STEP): 
    try:
        x_i = data[i:i+WINDOW]
        y_i = data[i+WINDOW+FORECAST]  

        last_close = x_i[WINDOW-1]
        next_close = y_i

        if last_close < next_close:
            y_i = [1, 0]
        else:
            y_i = [0, 1] 

    except Exception as e:
        print e
        break

    X.append(x_i)
    Y.append(y_i)

X = [(np.array(x) - np.mean(x)) / np.std(x) for x in X] # comment it to remove normalization
X, Y = np.array(X), np.array(Y)

X_train, X_test, Y_train, Y_test = create_Xt_Yt(X, Y)